In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/Milwaukee-WIch12.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592552 entries, 0 to 592551
Data columns (total 11 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   Lon                                                    592267 non-null  float64
 1   Lat                                                    592267 non-null  float64
 2   Time                                                   592552 non-null  object 
 3   NR5G_RF_RSRP                                           104748 non-null  float64
 4   NR5G_RF_SINR                                           104776 non-null  float64
 5   NR5G_Throughput_PDSCH_TP                               104585 non-null  float64
 6   LTE_L1_RF_RSRP                                         91922 non-null   float64
 7   LTE_L1_RF_RS_SINR                                      91922 non-null   float64
 8   LTE_Data_Throughput_Downlink_All_P

In [ ]:
df['Time'] = pd.to_datetime(df['Time'])

In [ ]:
df = df.dropna(subset=['Lat', 'Lon'])

In [ ]:
df['TimeSec'] = df['Time'].dt.floor('1s')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 592267 entries, 6 to 592551
Data columns (total 12 columns):
 #   Column                                                 Non-Null Count   Dtype         
---  ------                                                 --------------   -----         
 0   Lon                                                    592267 non-null  float64       
 1   Lat                                                    592267 non-null  float64       
 2   Time                                                   592267 non-null  datetime64[ns]
 3   NR5G_RF_RSRP                                           104736 non-null  float64       
 4   NR5G_RF_SINR                                           104762 non-null  float64       
 5   NR5G_Throughput_PDSCH_TP                               104571 non-null  float64       
 6   LTE_L1_RF_RSRP                                         91909 non-null   float64       
 7   LTE_L1_RF_RS_SINR                                      91909 

In [ ]:
def identify_rat(row):
    if pd.notna(row['NR5G_RF_RSRP']):
        return '5G'
    elif pd.notna(row['LTE_L1_RF_RSRP']):
        return 'LTE'
    else:
        return 'N/a'

df['RAT'] = df.apply(identify_rat, axis=1)
df = df[df['RAT'] != 'N/a']

In [ ]:
cols = [
    'Time', 'TimeSec', 'Lat', 'Lon', 'RAT',
    'NR5G_RF_RSRP', 'NR5G_RF_SINR', 'NR5G_Throughput_PDSCH_TP',
    'LTE_L1_RF_RSRP', 'LTE_L1_RF_RS_SINR',
    'LTE_Data_Throughput_Downlink_All_PDSCH_PDSCH_TP_Total'
]
df = df[cols]

In [ ]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113861 entries, 6 to 592550
Data columns (total 11 columns):
 #   Column                                                 Non-Null Count   Dtype         
---  ------                                                 --------------   -----         
 0   Time                                                   113861 non-null  datetime64[ns]
 1   TimeSec                                                113861 non-null  datetime64[ns]
 2   Lat                                                    113861 non-null  float64       
 3   Lon                                                    113861 non-null  float64       
 4   RAT                                                    113861 non-null  object        
 5   NR5G_RF_RSRP                                           104736 non-null  float64       
 6   NR5G_RF_SINR                                           104760 non-null  float64       
 7   NR5G_Throughput_PDSCH_TP                               104570

In [ ]:
df = df.sort_values(by='Time')
pointsDF = df.groupby(['TimeSec','RAT']).tail(1).reset_index(drop=True)

In [ ]:
pointsDF['LTE_Throughput'] = pointsDF['LTE_Data_Throughput_Downlink_All_PDSCH_PDSCH_TP_Total']

pointsDF['RSRP'] = np.where(pointsDF['RAT']=='5G', pointsDF['NR5G_RF_RSRP'], pointsDF['LTE_L1_RF_RSRP'])
pointsDF['SINR'] = np.where(pointsDF['RAT']=='5G', pointsDF['NR5G_RF_SINR'], pointsDF['LTE_L1_RF_RS_SINR'])
pointsDF['Throughput'] = np.where(pointsDF['RAT']=='5G', pointsDF['NR5G_Throughput_PDSCH_TP'], pointsDF['LTE_Throughput'])

In [ ]:
pointsDF

,Time,TimeSec,Lat,Lon,RAT,NR5G_RF_RSRP,NR5G_RF_SINR,NR5G_Throughput_PDSCH_TP,LTE_L1_RF_RSRP,LTE_L1_RF_RS_SINR,LTE_Data_Throughput_Downlink_All_PDSCH_PDSCH_TP_Total,LTE_Throughput,RSRP,SINR,Throughput
0,2025-04-13 13:38:04,2025-04-13 13:38:04,43.055851,-88.471715,LTE,NaN,NaN,NaN,-114.38,1.50,NaN,NaN,-114.38,1.50,NaN
1,2025-04-13 13:38:06,2025-04-13 13:38:06,43.055851,-88.471715,LTE,NaN,NaN,NaN,-114.25,0.50,NaN,NaN,-114.25,0.50,NaN
2,2025-04-13 13:38:07,2025-04-13 13:38:07,43.055851,-88.471715,LTE,NaN,NaN,NaN,-114.81,1.00,NaN,NaN,-114.81,1.00,NaN
3,2025-04-13 13:38:08,2025-04-13 13:38:08,43.055851,-88.471715,LTE,NaN,NaN,NaN,-114.06,-2.70,NaN,NaN,-114.06,-2.70,NaN
4,2025-04-13 13:38:09,2025-04-13 13:38:09,43.055851,-88.471715,LTE,NaN,NaN,NaN,-114.75,0.90,NaN,NaN,-114.75,0.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113856,2025-04-16 21:26:26,2025-04-16 21:26:26,43.061773,-88.146049,5G,-112.70,7.80,0.056,-115.25,-2.00,NaN,NaN,-112.70,7.80,0.056
113857,2025-04-16 21:26:27,2025-04-16 21:26:27,43.061717,-88.146056,5G,-112.43,8.16,0.033,-114.89,0.23,NaN,NaN,-112.43,8.16,0.033
113858,2025-04-16 21:26:28,2025-04-16 21:26:28,43.061668,-88.146062,5G,-112.17,6.62,0.090,-113.46,-0.18,0.0,0.0,-112.17,6.62,0.090
113859,2025-04-16 21:26:29,2025-04-16 21:26:29,43.061631,-88.146068,5G,-111.46,7.77,0.040,-114.36,-0.84,NaN,NaN,-111.46,7.77,0.040


In [ ]:
pointsDF = pointsDF[[
    'TimeSec',
    'Time',
    'Lat',
    'Lon',
    'RAT',
    'RSRP',
    'SINR',
    'Throughput'
]]
pointsDF

,TimeSec,Time,Lat,Lon,RAT,RSRP,SINR,Throughput
0,2025-04-13 13:38:04,2025-04-13 13:38:04,43.055851,-88.471715,LTE,-114.38,1.50,NaN
1,2025-04-13 13:38:06,2025-04-13 13:38:06,43.055851,-88.471715,LTE,-114.25,0.50,NaN
2,2025-04-13 13:38:07,2025-04-13 13:38:07,43.055851,-88.471715,LTE,-114.81,1.00,NaN
3,2025-04-13 13:38:08,2025-04-13 13:38:08,43.055851,-88.471715,LTE,-114.06,-2.70,NaN
4,2025-04-13 13:38:09,2025-04-13 13:38:09,43.055851,-88.471715,LTE,-114.75,0.90,NaN
...,...,...,...,...,...,...,...,...
113856,2025-04-16 21:26:26,2025-04-16 21:26:26,43.061773,-88.146049,5G,-112.70,7.80,0.056
113857,2025-04-16 21:26:27,2025-04-16 21:26:27,43.061717,-88.146056,5G,-112.43,8.16,0.033
113858,2025-04-16 21:26:28,2025-04-16 21:26:28,43.061668,-88.146062,5G,-112.17,6.62,0.090
113859,2025-04-16 21:26:29,2025-04-16 21:26:29,43.061631,-88.146068,5G,-111.46,7.77,0.040


In [ ]:
pointsDF['Throughput'] = pointsDF['Throughput'].fillna(0)

/tmp/ipython-input-1716417707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['Throughput'] = pointsDF['Throughput'].fillna(0)


In [ ]:
pointsDF.isna().sum()

,0
TimeSec,0
Time,0
Lat,0
Lon,0
RAT,0
RSRP,0
SINR,25
Throughput,0


In [ ]:
pointsDF['SINR'] = pointsDF.groupby('RAT')['SINR'].transform(
    lambda x: x.fillna(x.median())
)


/tmp/ipython-input-2953201941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['SINR'] = pointsDF.groupby('RAT')['SINR'].transform(


In [ ]:
pointsDF.to_csv('/content/cleaned_points.csv', index=False)

In [ ]:
pointsDF.isna().sum()

,0
TimeSec,0
Time,0
Lat,0
Lon,0
RAT,0
RSRP,0
SINR,0
Throughput,0


In [ ]:
pointsDF['z_RSRP'] = pointsDF.groupby('RAT')['RSRP'].transform(
    lambda x: (x - x.mean()) / x.std()
)

pointsDF['z_SINR'] = pointsDF.groupby('RAT')['SINR'].transform(
    lambda x: (x - x.mean()) / x.std()
)

pointsDF['z_Throughput'] = pointsDF.groupby('RAT')['Throughput'].transform(
    lambda x: (x - x.mean()) / x.std()
)

/tmp/ipython-input-1912268150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['z_RSRP'] = pointsDF.groupby('RAT')['RSRP'].transform(
/tmp/ipython-input-1912268150.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['z_SINR'] = pointsDF.groupby('RAT')['SINR'].transform(
/tmp/ipython-input-1912268150.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [ ]:
pointsDF

,TimeSec,Time,Lat,Lon,RAT,RSRP,SINR,Throughput,z_RSRP,z_SINR,z_Throughput
0,2025-04-13 13:38:04,2025-04-13 13:38:04,43.055851,-88.471715,LTE,-114.38,1.50,0.000,-0.824876,-0.514209,-0.466002
1,2025-04-13 13:38:06,2025-04-13 13:38:06,43.055851,-88.471715,LTE,-114.25,0.50,0.000,-0.813266,-0.647616,-0.466002
2,2025-04-13 13:38:07,2025-04-13 13:38:07,43.055851,-88.471715,LTE,-114.81,1.00,0.000,-0.863280,-0.580912,-0.466002
3,2025-04-13 13:38:08,2025-04-13 13:38:08,43.055851,-88.471715,LTE,-114.06,-2.70,0.000,-0.796297,-1.074516,-0.466002
4,2025-04-13 13:38:09,2025-04-13 13:38:09,43.055851,-88.471715,LTE,-114.75,0.90,0.000,-0.857921,-0.594253,-0.466002
...,...,...,...,...,...,...,...,...,...,...,...
113856,2025-04-16 21:26:26,2025-04-16 21:26:26,43.061773,-88.146049,5G,-112.70,7.80,0.056,-1.307488,-0.754981,-0.500929
113857,2025-04-16 21:26:27,2025-04-16 21:26:27,43.061717,-88.146056,5G,-112.43,8.16,0.033,-1.285356,-0.720866,-0.501023
113858,2025-04-16 21:26:28,2025-04-16 21:26:28,43.061668,-88.146062,5G,-112.17,6.62,0.090,-1.264043,-0.866804,-0.500790
113859,2025-04-16 21:26:29,2025-04-16 21:26:29,43.061631,-88.146068,5G,-111.46,7.77,0.040,-1.205844,-0.757824,-0.500994


In [ ]:
pointsDF['Performance'] = (
    0.4 * pointsDF['z_RSRP'] +
    0.4 * pointsDF['z_SINR'] +
    0.2 * pointsDF['z_Throughput']
)

/tmp/ipython-input-3735758578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['Performance'] = (


In [ ]:
def perf_label(value):
    if value < -1:
        return "Bad"
    elif value <= 1:
        return "Average"
    else:
        return "Great"

pointsDF['Perf_Label'] = pointsDF['Performance'].apply(perf_label)
pointsDF

/tmp/ipython-input-1107534633.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pointsDF['Perf_Label'] = pointsDF['Performance'].apply(perf_label)


,TimeSec,Time,Lat,Lon,RAT,RSRP,SINR,Throughput,z_RSRP,z_SINR,z_Throughput,Performance,Perf_Label
0,2025-04-13 13:38:04,2025-04-13 13:38:04,43.055851,-88.471715,LTE,-114.38,1.50,0.000,-0.824876,-0.514209,-0.466002,-0.628835,Average
1,2025-04-13 13:38:06,2025-04-13 13:38:06,43.055851,-88.471715,LTE,-114.25,0.50,0.000,-0.813266,-0.647616,-0.466002,-0.677553,Average
2,2025-04-13 13:38:07,2025-04-13 13:38:07,43.055851,-88.471715,LTE,-114.81,1.00,0.000,-0.863280,-0.580912,-0.466002,-0.670877,Average
3,2025-04-13 13:38:08,2025-04-13 13:38:08,43.055851,-88.471715,LTE,-114.06,-2.70,0.000,-0.796297,-1.074516,-0.466002,-0.841526,Average
4,2025-04-13 13:38:09,2025-04-13 13:38:09,43.055851,-88.471715,LTE,-114.75,0.90,0.000,-0.857921,-0.594253,-0.466002,-0.674070,Average
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113856,2025-04-16 21:26:26,2025-04-16 21:26:26,43.061773,-88.146049,5G,-112.70,7.80,0.056,-1.307488,-0.754981,-0.500929,-0.925173,Average
113857,2025-04-16 21:26:27,2025-04-16 21:26:27,43.061717,-88.146056,5G,-112.43,8.16,0.033,-1.285356,-0.720866,-0.501023,-0.902693,Average
113858,2025-04-16 21:26:28,2025-04-16 21:26:28,43.061668,-88.146062,5G,-112.17,6.62,0.090,-1.264043,-0.866804,-0.500790,-0.952497,Average
113859,2025-04-16 21:26:29,2025-04-16 21:26:29,43.061631,-88.146068,5G,-111.46,7.77,0.040,-1.205844,-0.757824,-0.500994,-0.885666,Average


In [ ]:
pointsDF.to_csv('/content/cleaned_points_with_lables.csv', index=False)

In [ ]:
!pip install folium

In [ ]:
def perf_to_color(value):
    if value < -1:
        return 'red'
    elif value <= 1:
        return 'orange'
    else:
        return 'green'

In [ ]:
import folium

In [ ]:
center_lat = pointsDF['Lat'].mean()
center_lon = pointsDF['Lon'].mean()

m_points = folium.Map(location=[center_lat, center_lon], zoom_start=12)

In [ ]:
for _, row in pointsDF.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lon']],
        radius=3,
        color=perf_to_color(row['Performance']),
        fill=True,
        fill_opacity=0.7,
        popup=(
            f"Time: {row['TimeSec']}<br>"
            f"RAT: {row['RAT']}<br>"
            f"RSRP: {row['RSRP']}<br>"
            f"SINR: {row['SINR']}<br>"
            f"Throughput: {row['Throughput']}<br>"
            f"Perf: {round(row['Performance'], 2)}"
        )
    ).add_to(m_points)

In [ ]:
m_points.save('/content/map_points.html')

In [ ]:
pathDF = pointsDF.sort_values(by='TimeSec')

In [ ]:
m_path = folium.Map(
    location=[pointsDF['Lat'].mean(), pointsDF['Lon'].mean()],
    zoom_start=12
)

In [ ]:
coords = pathDF[['Lat', 'Lon', 'Performance']].values

def perf_to_color(value):
    if value < -1:
        return 'red'
    elif value <= 1:
        return 'orange'
    else:
        return 'green'

for i in range(len(coords) - 1):
    lat1, lon1, perf1 = coords[i]
    lat2, lon2, perf2 = coords[i + 1]

    seg_color = perf_to_color((perf1 + perf2) / 2)

    folium.PolyLine(
        locations=[[lat1, lon1], [lat2, lon2]],
        color=seg_color,
        weight=4,
        opacity=0.9
    ).add_to(m_path)

In [ ]:
m_path.save('/content/map_path.html')

In [ ]:
!pip install keplergl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 78.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached jupyter_packaging-0.12.3-py3-none-any.whl.metadata (7.7 kB)
  Using cached comm-0.2.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached jupyter_lsp-2.3.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyterlab_server-2.28.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached json5-0.12.1-py3-none-any.whl.metadata (36 kB)
Using cached ipywidgets-8.1.8-py3-none-any.whl (139 kB)
Using cached jupyter_packaging-0.12.3-py3